# Practica 4

np.multiply hace multiplicacion elemento a elemento. Para hacer esto, no usar dot ni matmul

In [103]:
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import checkNNGradients as checkGradiente

In [104]:
weights = loadmat('ex4weights.mat')
theta1, theta2 = weights['Theta1'], weights['Theta2']

data = loadmat('ex4data1.mat')
y = data['y']
X = data['X']

In [105]:
y = np.ravel(y)
m=len(y)
X_1 = np.hstack([np.ones([m, 1]), X])

In [106]:
def sigmoide(z):
    return (1.0/(1.0+np.exp(-z)))

In [107]:
def forwardprop(X_1, theta1, theta2):
    a_1 = X_1;
    z_2 = np.matmul(a_1, theta1.T)
    a_2 = sigmoide(z_2)
    a_2 = np.hstack([np.ones([len(a_2), 1]), a_2])
    z_3 = np.matmul(a_2, theta2.T)
    a_3 = sigmoide(z_3)
    return a_1, z_2, a_2, z_3, a_3

In [108]:
a_1, z_2, a_2, z_3, a_3 = forwardprop(X_1, theta1, theta2)

In [109]:
print(a_3.shape)

(5000, 10)


In [110]:
def yOneHot(y, num_etiquetas):
    y = (y-1)
    y_onehot = np.zeros((m, num_etiquetas)) #Matriz 5000 x 10
    for i in range(m):
        y_onehot[i][y[i]] = 1
    return y_onehot

In [111]:
def coste(H, y, m):
    Term1 = y * np.log(H) 
    Term2 =  (1 - y) * np.log(1 - H)
    cost = ( 1 / m) * np.sum(-Term1 - Term2)
    return cost

In [112]:
y_ = yOneHot(y, 10)
cost = coste(a_3, y_,m)
print(cost)

0.2876291651613189


In [113]:
def coste_reg(theta1, theta2, coste, lamb, m):
    reg = (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2))) 
    coste_reg = coste + (reg* (lamb/(2*m)))
    return coste_reg

In [115]:
coste_reg = coste_reg(theta1, theta2, cost, 1, m)
print(coste_reg)

0.38376985909092365


In [122]:
def func_g(z):
    ones = np.ones(z.shape)
    a = sigmoide(z)
    return a*(ones-a)

In [125]:
def backprop(params_rn, num_entradas, num_ocultas, num_etiquetas, X, y, reg):
    Theta1 = np.reshape(params_rn[:num_ocultas * (num_entradas + 1)],(num_ocultas, (num_entradas + 1)))
    Theta2 = np.reshape(params_rn[num_ocultas * (num_entradas + 1):],(num_etiquetas, (num_ocultas + 1)))
    
    #Forward propagation
    a_1, z_2, a_2, z_3, a_3 = forwardprop(X_1, theta1, theta2)
    
    #Backward propagation
    sigma_3 = a_3 - y
    sigma_2 = np.matmul(theta2.T[1:, :], sigma_3.T) * func_g(z_2).T
    
    delta_1 = np.zeros(theta1.shape)
    delta_2 = np.zeros(theta2.shape)
    
    delta_1 = delta_1 + np.matmul(sigma_2, a_1)
    delta_2 = delta_2 + np.matmul(sigma_3.T, a_2)
    
    grad_1 = (1/len(y))*np.sum(delta_1)
    grad_2 = (1/len(y))*np.sum(delta_2)
    
    gradiente = np.concatenate((grad_1, grad_2), axis = None)
    
    m = len(y)
    cost = coste(a_3, y, m)
    
    return (cost, gradiente)

In [126]:
print(backprop(params_rn, 400, 25, 10, X_1, y_, 0))

(0.2876291651613189, array([-0.04277453,  0.06882196]))


In [127]:
params_rn =  np.concatenate((theta1, theta2), axis = None)
print(checkGradiente.checkNNGradients(backprop, 0))

ValueError: operands could not be broadcast together with shapes (5000,10) (5,3) 